# Formally verified Adder

Simple 8 bit adder function using bit masking and shifting 

Create a loop to check result of all inputs

In [1]:
# Function Definition of an variable-bit adder
def adder_var_bit(a, b, length):
    #create variables necessary to catch all conditions.
    carryFlag = 0
    zeroFlag = 0
    overflowFlag = 0
    result = 0

    # Extract bits for arithmetic use.
    for i in range(length):
        # curr_a and curr_b is a 1 or 0 value at a position i in the 8 bit value.
        curr_a = (a >> i) & 1
        curr_b = (b >> i) & 1
        
        # We or the bits together to create binary addition making sure to include a
        # potential carry value, and place it in the sum_bit variable.
        sum_bit = curr_a ^ curr_b ^ carryFlag

        # We then set the carryFlag using boolean logic on the other two bits as well as 
        # check the carryFlag is not already active.
        carryFlag = (curr_a & curr_b) | (curr_a & carryFlag) | (curr_b & carryFlag)
        
        # Then we append the current working bit onto the result value.
        result |= (sum_bit << i)
        
        # Check for overflow on final bit calculation
        if (carryFlag & 1) != ((result >> length-1) & 0):
            overflowFlag = 1
    return result, overflowFlag

In [2]:
# Example
a = 0b10000000
b = 0b10000001
add, overflowFlag = adder_var_bit(a, b, 8)

In [3]:
print("Binary Sum:", bin(add)[2:].zfill(8))
print("Decimal Sum", add)
print("Overflow Flag:", overflowFlag)

Binary Sum: 00000001
Decimal Sum 1
Overflow Flag: 1


# MiniFloat Adder

Using an 8 bit FP value that has 1 sign bit, 4 exponent bits, and 3 mantissa bits

How is it calculated??

X<e,m> =  | E = 0,     (-1)^S * 2^(1-B) * (0 + F * 2^-m) | (denormal)

otherwise | (-1)^S * 2^(E-B) * (1 + F * 2^-m) | (normal)

where B = 2^(e - 1) - 1

Example: if we have the 8-bit value of 0 0110 010

sign = negative

exponent in decimal = 5

mantissa in decimal = 2

B = 2^4 - 1 = 7

(denormal calculation) -> (-1)^0 * 2^(1-7) * (0 + 2*2^-3) = 1 * 0.015625 * 0.25 = 3.90625e-3

(normal calculation) -> (-1)^0 * 2^(6-7) * (1 + 2*2^-3) = 1 * 0.5 + 0.125 = 6.25e-1

Minimum denormal values X<e,m> = 0 0000 001 -> 1 * 2^-6 * (1*2^-3) = 2^-9.

Maximum positive normal is then: X<e,m> = 0 1111 111 -> 1 * 2^(15-7) * (1 + 7*2^-3) = 480

If denormal numbers are used then all values where the mantissa is equal to 0 tend to 0.

0 1111 000 -> 0

1 1111 000 -> 0

0 0000 000 -> 0

1 0000 000 -> -0

Use normal numbers? What is the purpose of denormal numbers (subnormal etc)

Computer Arithmetic https://www.amazon.com.au/Digital-Arithmetic-Kaufmann-Computer-Architecture/dp/1558607986
Patterson and hennerson computer organisation and design

In [4]:
def mf43adder(a, b):
    carryFlag = 0
    overflowFlag = 0
    zeroFlag = 0
    result = 0;

    e = 4
    m = 3
    B = 2**(e-1)-1
    
    F_a = a & 0b111
    F_b = b & 0b111

    shift_a = a >> 3
    shift_b = b >> 3

    E_a = shift_a & 0b1111
    E_b = shift_b & 0b1111

    E_diff = E_a - E_b

    if (E_diff > 0):
        F_b <<= E_diff
        E_b += E_diff

    elif (E_diff < 0):
        E_diff = -E_diff
        F_a <<= E_diff
        E_a += E_diff

    sum_F = F_a + F_b
    sum_E = E_a
    if sum_F >= 8:
        sum_F &= 0b111  # Keep only the lower 3 bits
        sum_E += 1  # Increase exponent
    
    result = (-1)**0 * 2**(sum_E-B) * (1+sum_F*2**(-m))
    return result

In [5]:
a = 0b00011001
b = 0b00101010

print("mf43adder function calculation:", mf43adder(a,b))

mf43adder function calculation: 0.4375


In [6]:
a = (-1)**0 * 2**(3-7) * (1+1*2**(-3))
print("A Input:", a)
b = (-1)**0 * 2**(5-7) * (1+2*2**(-3))
print("B Input:", b)
c = a + b
print("Final floating point value:", c)

A Input: 0.0703125
B Input: 0.3125
Final floating point value: 0.3828125


### Clearly these do not match. Further testing/understanding required on method of adding.

## Kulisch Accumulator 

A Kulisch accumulator is a type of accumulator designed to minimize round-off errors in numerical computations, particularly in floating-point arithmetic. It was developed by mathematician Peter Kulisch in the late 1980s.

In numerical computations, especially those involving floating-point arithmetic, round-off errors can accumulate and lead to inaccuracies in the results. The Kulisch accumulator aims to mitigate these errors by using extended precision arithmetic and employing a method called "compensation" or "renormalization" to reduce the accumulation of errors.

The basic idea behind the Kulisch accumulator is to perform arithmetic operations in a higher precision than the final result and then adjust the result to the desired precision while minimizing the accumulation of round-off errors. This approach can improve the accuracy of numerical computations, especially in iterative algorithms where errors can propagate over multiple computations.

## Next Steps

Need to continue working on these applications to ensure we can achieve correct values, then further apply it to multiplication. Once we have this a preliminary attempt at verification should be made before moving onto the the Block MiniFloat version. From there we can attempt take what we have learned and build a FMA which we can verify.

## Floating Point Arithmetic

https://docs.oracle.com/cd/E19957-01/806-3568/ncg_goldberg.html

#### Rounding Error types:
ULPS - Units in the last place. 
When a calculation results in a value such $.0314159$ but is then represented as $3.14\times10^0$, we say that this representation is in error by $.159$ ULP. Essentially if the value were to be computed to infinite precision, the difference between the inifite precision value and the representation is the difference from the last incorrect signiifcant value.

Relative Error - Simply the difference between the two numbers divided by the real number. Using our example above we see that we get:

$$\frac{3.14159 - 3.14\times10^0}{3.14159} = \frac{0.00159}{3.14159} = 0.0005$$

Since their is a range of numbers that the floating point values can be based on the significand and exponents widths, we see that the range of error that can arise is between two values: $$\frac{1}{2}\beta^{-p}\leq\frac{1}{2}ulp\leq\frac{\beta}{2}\beta^{-p}$$

In particular we are might be concerned with when the relative error corresponds to $.5 ulp$. This can vary by a factor of $\beta$, and is commonly referred to as the $wobble$ factor. Setting $\epsilon$ to the largest boundary in the above inequality, we can say that the a real number rounded to the closest floating point number, the relative error is always bounded by the exponent. This is referred to as $\textit{machine epsilon}$

### Theorem 1:

Using a floating-point format with parameters $\beta$ and p, and computing differences using p digits, the relative error of the result can be as large as $\beta$ - 1

### Proof 1

A relative error of $\beta - 1$ in the expression $x - y$ occurs when $x = 1.00...0$ and $y =  .\rho\rho...\rho$, where $\rho = \beta - 1$. Here $y$ has p digits (all equal to $\rho$). The exact difference is then $x-y = \beta^{-p}$. However, critically, when working with p digits, the rightmost digit of $y$ gets shifted off, and so the computed difference is $\beta^{-p+1}$. 
Thus the error is: $$\beta^{-p} - \beta^{-p+1} = \beta^{-p}(\beta - 1)$$ 
and the relative error is: $$\frac{\beta^{-p}(\beta - 1)}{\beta^{-p}} = \beta - 1$$

###### 